#### Agent、AgentExecutor的创建

##### 传统方式
单工具使用

- 需求：今天北京的天气怎么样?
- 使用Tavily搜索工具
  - Tavily的搜索API是一个专门为人工智能Agent(llm)构建的搜索引擎，可以快速提供实时、准确和真实的结果。
  - LangChain 中有一个内置工具，可以轻松使用 `Tavily 搜索引擎`作为工具。
  - TAVILY_API_KEY申请：https://tavily.com/，注册账号并登录，创建 API 密钥。

##### 方式1：ReAct模式

- AgentType是 <strong style='color:red'>ZERO_SHOT_REACT_DESCRIPTION</strong>

In [3]:
import os

from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
import dotenv

dotenv.load_dotenv()
# 1. 设置 API 密钥
# os.environ["TAVILY_API_KEY"] = "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"

# 2. 初始化搜索工具
search = TavilySearchResults(max_results=3)

# 3. 创建Tool的实例 （本步骤可以考虑省略，后续直接使用[search]替换[search_tool]
search_tool = Tool(
    name="Search",
    func=search,
    description="用于搜索互联网上的信息"
)

# 4. 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))

# 5. 创建 AgentExecutor
agent_executor = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 5. 测试查询
query = "今天北京的天气怎么样？"
result = agent_executor.invoke(query)
print(f"查询结果: {result}")

/var/folders/88/h2503n817r15dm_yghgn_vcm0000gn/T/ipykernel_13193/910653952.py:26: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(




> Entering new AgentExecutor chain...
我需要查找今天北京的天气信息。  
Action: Search  
Action Input: "今天 北京 天气"  
Observation: [{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511', 'content': '|  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 26.7℃ | 30.6℃ | 33.8℃ | 32.6℃ | 30.5℃ | 28.4℃ | 26.3℃ | 24.2℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 3.2m/s | 2.2m/s | 2.5m/s | 3.3m/s | 2.4m/s | 2.4m/s | 2.8m/s | 2.1m/s |\n| 风向 | 西北风 | 东北风 | 西南风 | 西南风 | 西南风 | 西北风 | 东北风 | 东北风 | [...] |  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 25.7℃ | 25.8℃ | 26.8℃ | 26.8℃ | 24℃ | 23.4℃ | 22.7℃ | 22.2℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm |\n| 风速 | 3m/

##### 方式2：FUNCATION_CALL模式

- AgentType是 <strong style='color:red'>OPENAI_FUNCTIONS</strong>

提示：只需要修改前面代码中的initialize_agent中的agent参数值。

In [7]:
# 1.导入相关包
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool, StructuredTool
from langchain_community.tools.tavily_search import TavilySearchResults
import langchain

langchain.debug = False

# 3. 初始化搜索工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    name="Search",
    func=search,
    description="用于搜索互联网上的信息"
)
llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))

# 5. 创建AgentExecutor执行器对象
agent_executor = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  #唯一变化
    verbose=True
)

# 6. 测试查询
query = "今天北京的天气怎么样？"
result = agent_executor.invoke(query)
print(f"查询结果: {result}")



> Entering new AgentExecutor chain...

Invoking: `Search` with `北京今天的天气`


[{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511', 'content': '中国气象局-天气预报-城市预报 - 天气预报 城市预报 | 气温 | 28.9℃ | 32.8℃ | 27.8℃ | 26.8℃ | 24℃ | 23.2℃ | 26.7℃ | 30.6℃ | | 气温 | 26.7℃ | 30.6℃ | 33.8℃ | 32.6℃ | 30.5℃ | 28.4℃ | 26.3℃ | 24.2℃ | | 气温 | 27.1℃ | 29.9℃ | 32.8℃ | 31.5℃ | 29℃ | 26.9℃ | 24.3℃ | 24.2℃ | | 气温 | 25.7℃ | 25.8℃ | 26.8℃ | 26.8℃ | 24℃ | 23.4℃ | 22.7℃ | 22.2℃ | | 气温 | 24.3℃ | 26.2℃ | 28.8℃ | 28.4℃ | 26.4℃ | 25.7℃ | 23.8℃ | 23.2℃ | | 气温 | 24.6℃ | 27.8℃ | 29.8℃ | 29.4℃ | 27.1℃ | 24.2℃ | 22.2℃ | 22.2℃ |', 'score': 0.98537}, {'title': '预报- 北京', 'url': 'https://www.weather.com.cn/weather/101010100.shtml', 'content': '北京天气预报,北京7天天气预报,北京15天天气预报,北京天气查询 更多 :   北京 上海 天津 重庆 :   北京; "北京")上海; "上海")天津; "天津")重庆; "重庆")黑龙江; "黑龙江")吉林; "吉林")辽宁; "辽宁")内蒙古; "内蒙古")河北; "河北")山西; "山西")陕西; "陕西")山东; "山东")新疆; "新疆")西藏; "西藏")青海; "青海")甘肃; "甘肃")宁夏; "宁夏")河南; "河南")江苏; "江苏")湖北; "湖北")浙江; "浙江")安徽; "安徽")福建; "福建")江西

#### 案例2：多工具使用

- 需求：计算特斯拉当前股价是多少？比去年上涨了百分之几？（使用PythonREPL的run方法）
- 多个（两个）工具的选择

##### 方式1：ReAct 模式

- AgentType是 <strong style=color:red>ZERO_SHOT_REACT_DESCRIPTION</strong>

In [12]:
# 1.导入相关依赖
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL

# 3.定义搜索工具
search = TavilySearchResults(max_results=3)

search_tool = Tool(
    name="Search",
    func=search.run,
    description="用于搜索互联网上的信息，特别是股票价格和新闻"
)

# 4.定义计算工具
python_repl = PythonREPL()  # LangChain封装的工具类可以进行数学计算

calc_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算，例如计算百分比变化"
)

# 5. 定义LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))

# 6. 创建AgentExecutor执行器对象
agent_executor = initialize_agent(
    tools=[search_tool, calc_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 7. 测试股票价格查询
query = "比亚迪当前股价是多少？比去年上涨了百分之几？"
result = agent_executor.invoke(query)
print(f"查询结果: {result}")



> Entering new AgentExecutor chain...
我需要首先查找比亚迪当前的股价，以便与去年的股价进行比较，从而计算出百分比的变化。  
Action: Search  
Action Input: '比亚迪当前股价'  
Observation: [{'title': '比亚迪', 'url': 'https://gu.qq.com/sz002594', 'content': '6\n\n夏佐全\n\n0.68%\n\n自然人股\n\n2,066\n\n2,066\n\n未变\n\n  \n\n\\--\n\n7\n\n王念强\n\n0.60%\n\n自然人股\n\n1,830\n\n1,830\n\n未变\n\n  \n\n\\--\n\n8\n\n中国工商银行股份有限公司－...\n\n0.49%\n\n境内法人股\n\n1,479\n\n1,557\n\n减持\n\n  \n\n78\n\n9\n\n中央汇金资产管理有限责任公司\n\n0.39%\n\n国有股\n\n1,198\n\n1,198\n\n未变\n\n  \n\n\\--\n\n10\n\n中国建设银行股份有限公司－...\n\n0.34%\n\n境内法人股\n\n1,036\n\n\\--\n\n新增\n\n  \n\n1,036\n\n自选股提醒\n\n查看历史提醒消息比亚迪(002594)当前股价：363.22\n\n出现以下情况时，您会收到QQ消息提醒Image 29\n\nQQ消息是指当您开着QQ聊天工具时，电脑右下角QQ弹出的小消息提示框\n\n股价提醒\n\n开盘后涨跌幅超过\n\n%\n\n(每日最多一次)\n\n股价上涨达到或高于\n\n元\n\n(即比现价上涨超过%）\n\n股价下跌达到或低于\n\n元\n\n(即比现价下跌超过%）\n\n资讯提醒\n\n当有最新公司公告时当有最新研究报告时 [...] 融资融券\\-1.18%\n   转融券标的\\-1.19%\n   深股通\\-1.09%\n   纳入富时罗素\\-0.73%\n   AH股\\-0.36%\n   锂电池概念\\-1.00%\n\n地域：\n\n   广东\\-1.10%\n\n五档盘口实时资金流向五档盘口\n\n委比\n\n\\-27.32%\n\n委差\n\n\\-1

In [17]:
# 1.导入相关依赖
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL

# 3.定义搜索工具
search = TavilySearchResults(max_results=3)

search_tool = Tool(
    name="Search",
    func=search.run,
    description="用于搜索互联网上的信息，特别是股票价格和新闻"
)

# 4.定义计算工具
python_repl = PythonREPL()  # LangChain封装的工具类可以进行数学计算

calc_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算，例如计算百分比变化，请务必使用此工具"
)

# 5. 定义LLM
llm = ChatOpenAI(model="gpt-4", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))

# 6. 创建AgentExecutor执行器对象
agent_executor = initialize_agent(
    tools=[search_tool, calc_tool],
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True
)

# 7. 测试股票价格查询
# query = "特斯拉当前股价是多少？比去年上涨了百分之几？"
# result=agent_executor.invoke(query)
result = agent_executor.invoke({"input": "比亚迪当前股价是多少？比去年上涨了百分之几？"})
print(f"查询结果: {result}")



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'tool_input': '当前比亚迪股票价格'}`


[{'title': '比亚迪(002594)股票最新价格行情,实时走势图,股价分析预测 - 英为财情', 'url': 'https://cn.investing.com/equities/byd-a', 'content': '施耐德电气\n\n214.80\n\nSCHN\n\n\\-0.30%\n\nStellantis NV\n\n8.757\n\nSTLAM\n\n+0.53%\n\nNovo Nordisk B\n\n444.3\n\nNOVOb\n\n+3.59%\n\n宁德时代\n\n248.27\n\n300750\n\n+1.51%\n\n常见问题\n----\n\n### 比亚迪(002594)现在的股价是多少？\n\n比亚迪现在的股价报363.22\n\n### 比亚迪的股票在哪间交易所挂牌交易？\n\n比亚迪的股票在深圳證券交易所挂牌交易。\n\n### 比亚迪的股票代码是什么？\n\n比亚迪的股票代码是“002594。”\n\n### 比亚迪有分红吗？当前的股息收益率是多少？\n\n比亚迪的股息收益率是0.85%。\n\n### 比亚迪的市值是多少？\n\n截至今天，比亚迪的市值是1.10T。\n\n### 比亚迪的每股收益(EPS)是多少？\n\n比亚迪的每股收益(EPS)是15.38。\n\n### 比亚迪的下一财报日是哪天？\n\n比亚迪会在2025年8月31日公布下一份财报。\n\n查找股票交易商\n\n货币\n\n指数\n\n期货\n\n股票\n\n1日\n\n1周\n\n1个月\n\n6个月\n\n1年\n\n5年\n\n最大值 [...] Aaa Aaaaaaaa A\n\n13.74\n\n+55.97%\n\n21.43\n\nAaaaaa Aa Aaa\n\n3.22\n\n+54.88%\n\n4.99\n\nAaaaaaaaaaaaaaa\n\n2.280\n\n+54.00%\n\n3.51\n\nAaaa Aaaaa\n\n13.89\n\n+53.92%\n\n21.38\n\nAaaaaa\n\n57.21\n\n

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null. (request id: 2025080711221688835002361W9Uu6c) (request id: 20250807112216781565335pzBwtWkL) (request id: B20250807112203359127963q7JCC4xx) (request id: 202508071122023381656211T9Szrv)", 'type': 'openai_error', 'param': '', 'code': '<nil>'}}

#### 案例3：自定义函数与工具

需求：计算3的平方，Agent自动调用工具完成

In [18]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI
import langchain


# 1. 定义工具 - 计算器（要求字符串输入）
def simple_calculator(expression: str) -> str:
    """
    基础数学计算工具，支持加减乘除和幂运算
    参数:
        expression: 数学表达式字符串，如 "3+5" 或 "2**3"
    返回:
        计算结果字符串或错误信息
    """
    print(f"\n[工具调用] 计算表达式: {expression}")

    print("只因为在人群中多看了你一眼，确认下你调用了我^_^")
    return str(eval(expression))


# 2. 创建工具对象
math_calculator_tool = Tool(
    name="Math_Calculator",  # 工具名称（Agent将根据名称选择工具）
    func=simple_calculator,  # 工具调用的函数
    description="用于数学计算，输入必须是纯数学表达式（如'3+5'或'3**2'表示平方）。不支持字母或特殊符号"  # 关键：明确输入格式要求
)

# 3. 初始化大模型
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))

# 4. 初始化AgentExecutor（使用零样本React模式、增加超时设置）
agent_executor = initialize_agent(
    tools=[math_calculator_tool],  # 可用的工具列表
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 简单指令模式
    verbose=True  # 关键参数！在控制台显示详细的推理过程
)

# 5. 测试工具调用（添加异常捕获）
print("=== 测试：正常工具调用 ===")
response = agent_executor.invoke("计算3的平方")  # 向Agent提问
print("最终答案:", response)

=== 测试：正常工具调用 ===


> Entering new AgentExecutor chain...
我需要计算3的平方，这可以通过数学表达式3**2来实现。
Action: Math_Calculator
Action Input: 3**2
[工具调用] 计算表达式: 3**2
只因为在人群中多看了你一眼，确认下你调用了我^_^

Observation: 9
Thought:我现在知道最终答案
Final Answer: 9

> Finished chain.
最终答案: {'input': '计算3的平方', 'output': '9'}


###  通用方式

需求：今天北京的天气怎么样？？

#### 方式1：FUNCATION_CALL模式

In [23]:
# 1.导入相关包
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

# 2.定义搜索化工具
# ① 设置 TAVILY_API 密钥
# os.environ["TAVILY_API_KEY"] = "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"  # 需要替换为你的 Tavily API 密钥

# ② 定义搜索工具
search = TavilySearchResults(max_results=1)
search_tool = Tool(
    name="search_tool",
    func=search.run,
    description="用于搜索互联网上的信息"
)
tools = [search_tool]
# 3.自定义提示词模版
prompt = ChatPromptTemplate.from_messages([
    ("system", "您是一位乐于助人的助手，请务必使用 search_tool 工具来获取信息。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 4.定义LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

# 5.创建Agent对象
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 6.创建AgentExecutor执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 7.测试
agent_executor.invoke({"input": "今天北京的天气怎么样？?"})



> Entering new AgentExecutor chain...

Invoking: `search_tool` with `今天北京的天气`


[{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511', 'content': '中国气象局-天气预报-城市预报 - 天气预报 城市预报 | 气温 | 28.9℃ | 32.8℃ | 27.8℃ | 26.8℃ | 24℃ | 23.2℃ | 26.7℃ | 30.6℃ | | 气温 | 26.7℃ | 30.6℃ | 33.8℃ | 32.6℃ | 30.5℃ | 28.4℃ | 26.3℃ | 24.2℃ | | 气温 | 27.1℃ | 29.9℃ | 32.8℃ | 31.5℃ | 29℃ | 26.9℃ | 24.3℃ | 24.2℃ | | 气温 | 25.7℃ | 25.8℃ | 26.8℃ | 26.8℃ | 24℃ | 23.4℃ | 22.7℃ | 22.2℃ | | 气温 | 24.3℃ | 26.2℃ | 28.8℃ | 28.4℃ | 26.4℃ | 25.7℃ | 23.8℃ | 23.2℃ | | 气温 | 24.6℃ | 27.8℃ | 29.8℃ | 29.4℃ | 27.1℃ | 24.2℃ | 22.2℃ | 22.2℃ |', 'score': 0.98562}]今天北京的天气气温在24.3℃到32.8℃之间，相对较暖和。详细天气预报可以查看[中国气象局网站](https://weather.cma.cn/web/weather/54511)。

> Finished chain.


{'input': '今天北京的天气怎么样？?',
 'output': '今天北京的天气气温在24.3℃到32.8℃之间，相对较暖和。详细天气预报可以查看[中国气象局网站](https://weather.cma.cn/web/weather/54511)。'}

#### 方式2：REACT模式

体会：使用PromptTemplate，如何体现可以使用的工具、用户输入和agent_scratchpad。

In [25]:
# 1.导入相关包
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate

# 2.定义搜索化工具
search = TavilySearchResults(max_results=1)

# 3.自定义提示词模版
template = '''Answer the following questions as best you can. You have access to the following tools:
            {tools}

            Use the following format:

            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question

            Begin!

            Question: {input}
            Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

# 4.定义LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

# 5.创建Agent对象
agent = create_react_agent(llm, [search], prompt)

# 6.创建AgentExecutor执行器
agent_executor = AgentExecutor(agent=agent, tools=[search], verbose=True,handle_parsing_errors=True)

# 7.测试
agent_executor.invoke({"input": "今天9月3日北京会有阅兵吗?"})



> Entering new AgentExecutor chain...
考虑到今天是9月3日，并且可能有关于北京阅兵的信息，我需要确认是否有阅兵活动。  
Action: tavily_search_results_json  
Action Input: "北京 9月3日 阅兵 2023"  [{'title': '快讯：9月3日北京天安门广场将举行阅兵', 'url': 'https://www.gov.cn/yaowen/liebiao/202506/content_7029176.htm', 'content': '# å¿«è®¯ï¼\x9a9æ\x9c\x883æ\x97¥å\x8c\x97äº¬å¤©å®\x89é\x97¨å¹¿å\x9cºå°\x86ä¸¾è¡\x8cé\x98\x85å\x85µ **æ\x96°å\x8d\x8eç¤¾å¿«è®¯ï¼\x9a**è®°è\x80\x85ä»\x8eå\x9b½æ\x96°å\x8a\x9eæ\x96°é\x97»å\x8f\x91å¸\x83ä¼\x9aè\x8e·æ\x82\x89ï¼\x8c9æ\x9c\x883æ\x97¥ä¸\x8aå\x8d\x88ï¼\x8cå\x8c\x97äº¬å¤©å®\x89é\x97¨å¹¿å\x9cºå°\x86ä¸¾è¡\x8cçºªå¿µä¸\xadå\x9b½äººæ°\x91æ\x8a\x97æ\x97¥æ\x88\x98äº\x89æ\x9a¨ä¸\x96ç\x95\x8cå\x8f\x8dæ³\x95è¥¿æ\x96¯æ\x88\x98äº\x89è\x83\x9cå\x88©80å\x91¨å¹´å¤§ä¼\x9aï¼\x8cå\x8c\x85æ\x8b¬æ£\x80é\x98\x85é\x83¨é\x98\x9fã\x80\x82 ä¸»å\x8a\x9eå\x8d\x95ä½\x8dï¼\x9aå\x9b½å\x8a¡é\x99¢å\x8a\x9eå\x85¬å\x8e\x85ã\x80\x80è¿\x90è¡\x8cç»´æ\x8a¤å\x8d\x95ä½\x8dï¼\x9aä¸\xadå\x9b½æ\x94¿åº\x9cç½\x91è¿\x90è¡\x8cä¸\xadå¿\x83 ç\x89\x88æ\x9d\x83æ\x89

{'input': '今天9月3日北京会有阅兵吗?', 'output': '今天9月3日北京会有阅兵。'}

## Agent嵌入记忆组件

### 4.1 传统方式

比如：3的平方是多少？加5呢？ （通过两次对话实现）

举例：以REACT模式为例

In [24]:
# 1. 导入依赖包
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.memory import ConversationBufferMemory

# 2. 设置 API 密钥
# os.environ["TAVILY_API_KEY"] = "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"

# 3. 定义搜索工具
search_tool = TavilySearchResults(max_results=2)

# 4. 定义LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

# 5. 定义记忆组件(以ConversationBufferMemory为例)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 6. 创建 AgentExecutor
agent_executor = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


# 7. 测试对话
# 第一个查询
query1="北京明天的天气怎么样？"
result1 = agent_executor.invoke(query1)
print(f"查询结果: {result1}")

# print("\n=== 继续对话 ===")
query2="上海呢"
result2=agent_executor.invoke(query2)
print(f"分析结果: {result2}")

/var/folders/88/h2503n817r15dm_yghgn_vcm0000gn/T/ipykernel_13193/1250435939.py:21: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: 北京明天天气预报
```
Observation: [{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511', 'content': '|  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 25.7℃ | 25.8℃ | 26.8℃ | 26.8℃ | 24℃ | 23.4℃ | 22.7℃ | 22.2℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm |\n| 风速 | 3m/s | 2.3m/s | 2.6m/s | 3.3m/s | 1.9m/s | 3.3m/s | 2.6m/s | 2.4m/s |\n| 风向 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | 西南风 | [...] |  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |  |  |  |  |  |  |  |  |\n| 气温 | 26.7℃ | 30.6℃ | 33.8℃ | 32.6℃ | 30.5℃ | 28.4℃ | 26.3℃ | 24.2℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无